In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline


In [3]:
class NullCompare():
    def __init__(self, g):
        self.G = g
    def CalculateStats(self):
        C = np.mean(list(nx.clustering(self.G,weight=None).values()))
        d = nx.average_shortest_path_length(self.G,weight= None)



        return C,d
    def ERPlot(self):
        GN = len(self.G.nodes())
        max_L = GN*(GN-1)/2
        actual_L = len(self.G.edges())
        p = actual_L/max_L

        ER = nx.erdos_renyi_graph(GN, p, directed=False)

        self.plot_degree_dist(ER)
    def ERStats(self,iterations):
        clustering_ER = []
        short_path_ER = []
        GN = len(self.G.nodes())
        max_L = GN*(GN-1)/2
        actual_L = len(self.G.edges())
        p = actual_L/max_L
        for i in range(iterations): # 1000 is better

            ER = nx.erdos_renyi_graph(GN, p, directed=False)
            print("Iteration " + str(i))
            C_ER = np.mean(list(nx.clustering(ER).values()))
            if nx.is_connected(ER)==True:
                d_ER = nx.average_shortest_path_length(ER,weight=None)
            clustering_ER.append(C_ER)
            short_path_ER.append(d_ER)


        return clustering_ER,short_path_ER
    def ClusteringCoffPlot(self,clustering_ER, C):
        fig = plt.figure(figsize=(6,4))

        plt.boxplot(clustering_ER)
        plt.plot(2,C,'r',marker='+',markersize=15)
        ax = plt.gca()
        ax.set_xticks([1,2],labels=[1,2])
        plt.xlim([0.5,2.5])
        plt.show()
    def ShortPathPlot(self,short_path_ER,d):
        fig = plt.figure(figsize=(6,4))

        plt.boxplot(short_path_ER)
        plt.plot(2,d,'r',marker='+',markersize=15)
        ax = plt.gca()
        ax.set_xticks([1,2],labels=[1,2])
        plt.xlim([0.5,2.5])
        plt.show()

     # Calculating metrics for a Degree Preservation Graph
    def DPPlot(self):
        DP = self.G.copy() # the function edits the graph in place

        nx.double_edge_swap(DP,nswap=self.G.number_of_edges(),max_tries=1000000)
        self.plot_degree_dist(DP)
        self.plot_degree_dist(self.G)
    def DPStats(self,iterations):
        clustering_DP = []
        short_path_DP = []
        DP = self.G.copy()
        for i in range(iterations): # 1000 is better

            nx.double_edge_swap(DP,nswap=self.G.number_of_edges(),max_tries=1000000)
            print("Iteration " + str(i))
            C_DP = np.mean(list(nx.clustering(DP).values()))
            if nx.is_connected(DP)==True:
                d_DP = nx.average_shortest_path_length(DP,weight=None)
            clustering_DP.append(C_DP)
            short_path_DP.append(d_DP)

        print(np.mean(clustering_DP))
        print(np.std(clustering_DP))

        print(np.mean(short_path_DP))
        print(np.std(short_path_DP))
        # Function Defintions

    # Function to plot a degree distribution graph (P(k) vs k graph)
    def plot_degree_dist(self,G):

        degrees = [G.degree(n) for n in G.nodes()]
        kmin = min(degrees)
        kmax = max(degrees)

        if kmin>0:
            bin_edges = np.logspace(np.log10(kmin), np.log10(kmax)+1, num=20)
        else:
            bin_edges = np.logspace(0, np.log10(kmax)+1, num=20)
        density, _ = np.histogram(degrees, bins=bin_edges, density=True)

        fig = plt.figure(figsize=(6,4))

        log_be = np.log10(bin_edges)
        x = 10**((log_be[1:] + log_be[:-1])/2)
        plt.loglog(x, density, marker='o', linestyle='none')
        plt.xlabel(r"degree $k$", fontsize=16)
        plt.ylabel(r"$P(k)$", fontsize=16)

        ax = plt.gca()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')